<em style="text-align:center">Copyright Ariel Chitay</em>

# Importar librerías iniciales e instancia de modelo de chat

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.agents import load_tools,initialize_agent,AgentType,create_react_agent,AgentExecutor
f = open('OpenAI_key.txt')
api_key = f.read()
llm = ChatOpenAI(openai_api_key=api_key,temperature=0) #Recomendable temperatura a 0 para que el LLM no sea muy creativo, vamos a tener muchas herramientas a nuestra disposición y queremos que sea más determinista

: 

### 🔄 Preparación del LLM y configuración del agente

En esta celda realizamos las configuraciones iniciales para construir nuestro **agente inteligente** que traducirá consultas en lenguaje natural a SQL de forma automatizada.

**¿Qué está pasando?**

1. **Importamos LangChain y componentes clave**  
   - `ChatOpenAI`: Nos permite interactuar con un modelo GPT (en este caso GPT-4 o GPT-3.5).
   - `PromptTemplate`, `ChatPromptTemplate`, `SystemMessagePromptTemplate`, `HumanMessagePromptTemplate`: Estas clases nos permitirán definir las instrucciones (prompts) que guiarán al agente, como vimos cuando explicamos el rol de los prompts en la sesión.
   - `AgentType`, `create_react_agent`, `AgentExecutor`: Herramientas específicas de LangChain para crear agentes que utilizan el patrón **ReAct** (Razonamiento + Acción).

2. **Lectura de la clave API**  
   - Estamos cargando nuestra clave de OpenAI desde un archivo local `OpenAI_key.txt`.
   - Esto habilita la conexión segura al modelo GPT.

3. **Inicialización del modelo LLM**  
   - Creamos un modelo de lenguaje determinista (`temperature=0`) para que al interactuar con herramientas (por ejemplo, Wikipedia, base de datos SQL, etc.) sus decisiones sean consistentes y basadas en el contexto, tal como lo vimos cuando explicamos el concepto de **agentes con ReAct**.
   - Esta es una buena práctica para evitar que la creatividad del modelo introduzca variaciones no deseadas en tareas estructuradas como la generación de SQL.

🔗 Relación con lo visto:
- Estamos conectando lo aprendido de **Model IO** (input/output del LLM).
- Preparando la base para que el agente pueda usar herramientas y ejecutar acciones sobre datos (lo que más adelante conectaremos con la **Base de Datos Vectorial** y consultas SQL automáticas).


## 1.Conectamos a la BBDD SQL

In [9]:
import mysql.connector #pip install mysql-connector-python

In [6]:
f = open('password_sql.txt')
pass_sql = f.read()
# Configuración de la conexión a la base de datos
config = {
    'user': 'root',       
    'password': pass_sql, 
    'host': '127.0.0.1',         
    'database': 'world'          
}

In [10]:
# Conectar a la base de datos
conn = mysql.connector.connect(**config)
cursor = conn.cursor() 

### Paso 1: Conexión a la Base de Datos SQL

- Se utiliza la librería `mysql.connector` para conectar Python con la base de datos `world`.
- Se lee la contraseña de un archivo externo para proteger las credenciales.
- Se configura la conexión indicando usuario, contraseña, host y base de datos.
- Se crea la conexión y un cursor que permitirá hacer consultas SQL desde Python.

### Relación con el Agente

Esta conexión permite al agente consultar datos reales y no solo generar texto genérico. Así, las respuestas que genere estarán basadas en información directa de la base de datos.


# 2. Ejecutamos consulta manualmente (sin agentes Langchain)

In [11]:
# Definir la consulta manualmente: tengo una base de datos mysql en mi computadora local denominada "world" y una tabla "Country" 
#sobre la que quiero hacer la suma de la población en la columna "Population" para el continente Asia (columna "Continent")
query = """
    SELECT SUM(Population)
    FROM Country
    WHERE Continent = 'Asia';
    """

# Ejecutar la consulta
cursor.execute(query)
result = cursor.fetchone()

In [12]:
suma_poblacion = result[0] if result[0] is not None else 0
print(f"La suma de la población del continente Asia es: {suma_poblacion}")

La suma de la población del continente Asia es: 3705025700


## Explicación: Consulta manual sin Langchain

- Aquí ejecutamos una consulta SQL tradicional directamente sobre la base de datos MySQL.
- Definimos manualmente la consulta para obtener la suma de la población (`SUM(Population)`) del continente Asia.
- Usamos `cursor.execute()` para ejecutar la consulta y `fetchone()` para obtener el resultado.
- Finalmente, mostramos la suma de la población.

💡 Nota:
- Este es un ejemplo de cómo normalmente consultaríamos una base de datos sin ayuda de un agente.
- Luego veremos cómo un agente puede automatizar este proceso desde lenguaje natural.


## 3.Creamos el agente SQL 

- En esta sección conectamos todo lo aprendido: bases de datos + LLM + Agentes.
- Usamos `create_sql_agent` de Langchain para que el LLM pueda interactuar directamente con MySQL.
- Creamos una **cadena de conexión** (connection string) que contiene los datos de acceso a la base de datos.
- Creamos un objeto de base de datos (`SQLDatabase`) que será la interfaz entre el agente y MySQL.
- Inicializamos el agente con `create_sql_agent`, conectándolo al LLM y a la base de datos.
- Finalmente, hacemos la **primera pregunta en lenguaje natural**: "Dime la población total de Asia".
- Ahora el agente analiza la pregunta, genera el SQL por sí solo, ejecuta la consulta y devuelve el resultado.

Con esto ya tenemos funcionando un **agente SQL** capaz de razonar, consultar y responder sin que nosotros escribamos el SQL manualmente.


In [13]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.sql_database import SQLDatabase

In [14]:
# Crear una cadena de conexión a la base de datos MySQL
connection_string = f"mysql+mysqlconnector://{config['user']}:{config['password']}@{config['host']}/{config['database']}"

# Crear una instancia de la base de datos SQL
db = SQLDatabase.from_uri(connection_string)

In [15]:
agent = create_sql_agent(
    llm,
    db=db,
    verbose=True
)

### Explicación detallada de `create_sql_agent`

- `create_sql_agent()` es una función de **LangChain** que crea un agente especializado para interactuar con bases de datos SQL.
  
- Este agente es una pieza clave porque le da al LLM la capacidad de:
   - Analizar preguntas en lenguaje natural.
   - Generar automáticamente consultas SQL.
   - Ejecutarlas en la base de datos conectada.
   - Procesar y devolver la respuesta final al usuario de forma entendible.

---

#### Parámetros utilizados:

- `llm`: Este es el **Large Language Model** que usaremos como cerebro del agente. En este caso es el modelo de OpenAI que configuramos antes (`ChatOpenAI()`).

- `db=db`: Aquí le indicamos al agente a qué base de datos debe conectarse. Este `db` es la instancia creada con `SQLDatabase.from_uri()`, que contiene la conexión ya establecida.

- `verbose=True`: Este parámetro es muy útil para propósitos de enseñanza o depuración. Si lo activamos:
    - El agente nos mostrará paso a paso lo que está haciendo.
    - Veremos la consulta SQL generada automáticamente.
    - Podemos identificar mejor cómo el LLM está razonando antes de ejecutar en la base de datos.

---

#### Relación con el flujo general:

El agente creado con esta línea es el **puente** entre:
- Lo que el usuario pregunta en lenguaje natural.
- La base de datos SQL que contiene las respuestas.
- El proceso automático de generación y ejecución de consultas sin necesidad de programar SQL explícito.

A partir de este punto, solo necesitaremos usar `agent.invoke("pregunta")` para hacer preguntas al estilo humano, y el agente hará todo lo demás.
```python
agent.invoke("Dime la población total de Asia")
```


In [16]:
agent.invoke("Dime la población total de Asia")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: city, country, countrylanguageI should query the country table to get the population of Asia.
Action: sql_db_schema
Action Input: country
CREATE TABLE country (
	`Code` CHAR(3) NOT NULL DEFAULT '', 
	`Name` CHAR(52) NOT NULL DEFAULT '', 
	`Continent` ENUM('Asia','Europe','North America','Africa','Oceania','Antarctica','South America') NOT NULL DEFAULT 'Asia', 
	`Region` CHAR(26) NOT NULL DEFAULT '', 
	`SurfaceArea` FLOAT(10, 2) NOT NULL DEFAULT '0.00', 
	`IndepYear` SMALLINT, 
	`Population` INTEGER NOT NULL DEFAULT '0', 
	`LifeExpectancy` FLOAT(3, 1), 
	`GNP` FLOAT(10, 2), 
	`GNPOld` FLOAT(10, 2), 
	`LocalName` CHAR(45) NOT NULL DEFAULT '', 
	`GovernmentForm` CHAR(45) NOT NULL DEFAULT '', 
	`HeadOfState` CHAR(60), 
	`Capital` INTEGER, 
	`Code2` CHAR(2) NOT NULL DEFAULT '', 
	PRIMARY KEY (`Code`)
)ENGINE=InnoDB DEFAULT CHARSET=latin1

/*
3 rows from country table:
Code	Name	Continent	Region	SurfaceArea

{'input': 'Dime la población total de Asia',
 'output': 'The total population of Asia is 4,028,017,500'}

In [17]:
result = agent.invoke("Dime el promedio de la esperanza de vida por cada una de las regiones ordenadas de mayor a menor")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: city, country, countrylanguageI should query the schema of the country table to see if it contains information about regions and life expectancy.
Action: sql_db_schema
Action Input: country
CREATE TABLE country (
	`Code` CHAR(3) NOT NULL DEFAULT '', 
	`Name` CHAR(52) NOT NULL DEFAULT '', 
	`Continent` ENUM('Asia','Europe','North America','Africa','Oceania','Antarctica','South America') NOT NULL DEFAULT 'Asia', 
	`Region` CHAR(26) NOT NULL DEFAULT '', 
	`SurfaceArea` FLOAT(10, 2) NOT NULL DEFAULT '0.00', 
	`IndepYear` SMALLINT, 
	`Population` INTEGER NOT NULL DEFAULT '0', 
	`LifeExpectancy` FLOAT(3, 1), 
	`GNP` FLOAT(10, 2), 
	`GNPOld` FLOAT(10, 2), 
	`LocalName` CHAR(45) NOT NULL DEFAULT '', 
	`GovernmentForm` CHAR(45) NOT NULL DEFAULT '', 
	`HeadOfState` CHAR(60), 
	`Capital` INTEGER, 
	`Code2` CHAR(2) NOT NULL DEFAULT '', 
	PRIMARY KEY (`Code`)
)ENGINE=InnoDB DEFAULT CHARSET=latin1

/*
3 rows from c

In [18]:
# Mostrar el resultado
print(result["output"])

El promedio de la esperanza de vida por cada una de las regiones ordenadas de mayor a menor es:
1. Australia and New Zealand - 78.8
2. Nordic Countries - 78.33333
3. Western Europe - 78.25556
4. British Islands - 77.25
5. Southern Europe - 76.52857
6. North America - 75.82
7. Eastern Asia - 75.25
8. Caribbean - 73.05833
9. Central America - 71.025
10. South America - 70.94615


In [5]:
# Para utilizar few-shoots para las consultas SQL: https://python.langchain.com/v0.1/docs/use_cases/sql/agents/

Claro, aquí te lo dejo en formato Markdown limpio para que puedas copiarlo directo:

```markdown
### Ejemplos de consultas manuales adicionales

#### Prompt 1:
> ¿Cuál es la expectativa de vida promedio de todos los países de Europa?

```sql
SELECT AVG(LifeExpectancy) 
FROM country 
WHERE Continent = 'Europe';
```

---

#### Prompt 2:
> ¿Cuál es el país con mayor población en América del Sur?

```sql
SELECT Name, Population 
FROM country 
WHERE Continent = 'South America' 
ORDER BY Population DESC 
LIMIT 1;
```

---

#### Prompt 3:
> ¿Cuántos países tienen una forma de gobierno de tipo República?

```sql
SELECT COUNT(*) 
FROM country 
WHERE GovernmentForm LIKE '%Republic%';
```
```

Si quieres te hago en el siguiente mensaje otros 3 pero pensados ya en modo prompt para agentes con lenguaje natural ✨. ¿Te los hago?